# Creazione di un basic tokenizer

Come primo step scarico un testo da utilizzare 

In [2]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7c1cd843d990>)

Una volta scaricato il testo viene letto in formato testo in una stringa

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Obiettivo è quello di convertire questo testo nelle singole word e caratteri speciali che lo compongono in modo da poterli successivamente convertire in embeddings.

Ci sono diversi modi x suddividere un testo in tokens. Una possibilità è utilizzare regexp.

Ad esempio per suddividere sulla base del carattere "spazio"

In [4]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


OLtre che in base allo spazio si può suddividere anche con la punteggiatura

In [5]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


Da questa lista è necessario escludere il carattere spazio (" "). Questo dipende dall'applicazione che dovrà utilizzare i token (ad es. x il codice lo spazio è significativo)

In [6]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


Infine possiamo gestire anche altri caratteri di punteggiatura

In [7]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Una volta ottenuto il nostro rudimentale tokenizer, è possibile utilizzarlo per il testo inizialmente scaricato

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


Visualizzo i primi 30 token

In [9]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


# Da Stringa a Integer

Una volta ottenuti i token in formato stringa è necessario procedere con la conversione in formato numerico (Token ID). <BR> Questa conversione rappresenta un passaggio intermedio verso la rappresentazione "finale" dei token in embedding vectors.

Primo step è la creazione di un vocabolary che mappi i singoli token presenti nel testo di input in formato stringa ai rispettivi token id. <br>
A tal fine è necessario predere tutti i distinti token e sortarli alfabeticamente

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


Visualizzo i primi 20 token

In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


Prossimo step sarà utilizzare questa mappatura per convertire i token da formato stringa a corrispondente token id.

Prima di fare questo step, creiamo una classe "Tokenizer" che implementi le funzionalità di :
- encode (da stringa a id numerico)
- decode (da id numerico a stringa)

In [12]:
class SimpleTokenizerV1:
    
    def __init__(self, vocab):

        # Stores the vocabulary as a class attribute for access in the encode and decode methods 
        self.str_to_int = vocab            
        # Creates an inverse vocabulary that maps token IDs back to the original text tokens 
        self.int_to_str = {i:s for s,i in vocab.items()}   

    # Processes input text into token IDs
    def encode(self, text):       
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    # Converts token IDs back into text 
    def decode(self, ids):         #4
        text = " ".join([self.int_to_str[i] for i in ids]) 

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) 
        return text

In [13]:
tokenizer = SimpleTokenizerV1(vocab)
text = "It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
print(tokenizer.decode(ids))

It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


Questo tokenizer ha il limite che è in grado di gestire SOLO token che erano presenti nel vocabolary con il quale è stato creato (quindi nel flusso di input).<br>

Step successivo sarà implementare il tokenizier appena creato al fine di poter gestire anche token non presenti nel vocab (unkown token) o token speciali per aggiungere informazioni che potrebbero essere utili alla comprensione del testo (ad es. token boundaries)

Aggiungiamo quindi al volabolary:
- <|unk|> per gestire token non presenti nel training set
- <|endoftext|>  per separare testi non correlati (ad es diversi documenti non correlati)

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


Visualizzo gli ultimi 5 elementi del vocab

In [16]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


Creo una nuova versione della classe Tokenizer per gestire i nuovi token

In [17]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # Replaces unknown words by <|unk|> tokens 
        preprocessed = [item if item in self.str_to_int            
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # Replaces spaces before the specified punctuations 
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)    
        return text


Per testare il nuovo tokenizer crei un testo concatenando 2 frasi e inserendo lo special token <|endoftext|> tra i 2 testi

In [18]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


Procedo con la tokenizzazione del testo concatenato

In [19]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


Tra i token id ottenuti sono presenti sia <|endoftext|> (1130) che <|unk|> (1131).

Infatti provando a riconvertire i token id in testo viene perso il contenuto originale degli UNK

In [20]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Il tokenizer utilizzato da GPT (BPE) non utilizza UNK ma solo <|endoftext|>

# BPE con TIKTOKEN

E' necessario installare la libreria (pip install tiktoken)

In [21]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.9.0


In maniera analoga a quanto fatto precedentemente con il tokenizer "base"

In [22]:
tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [23]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


A differenza del tokenizer base precedente, BPE è in grado di gestire gli unknown token scomponendo i token non presenti fino al livello di singolo carattere

Una volta definito il tokenizer da usare, è necessario creare il dataset con :
- input
- target

in modo da consentire il training di un modello che preveda la parola successiva a partire da una serie di parole in input. <br>A tal fine andremo a creare un dataloader che utilizza sliding windows per creare inpu-output a partire dal nostro testo iniziale. A tal fine andiamo a tokenizzare il testo con BPE

In [24]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


Per consentire il test andiamo ad utilizzare solo i primi 50 tokens

In [25]:
enc_sample = enc_text[50:]

In [26]:
# The context size determines how many tokens are included in the input.

context_size = 4    
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


E' quindi possibile, partendo dall'input, creare le coppie input --> target dove il target è il testo di input shiftato di 1.<br>
In pratica alla sinistra della freccia c'è l'input passato al llm e a destra c'è l'output (il carattere che deve essere previsto a fronte dell'input fornito)

In [27]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


Convertendo da token id a stringa testuale

In [28]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


Una volta create le coppie di input/output è possibile utilizzarle per effettuare il training dell'llm.<br>
Per ragioni di efficienza però, prima di poter procedere con il training, è necessario uno step di conversione di input e target in formato Pytorch Tensor<br>
A tal fine utilizzeremo le classi Pytorch:
- dataset
- dataloader

Creiamo quindi una classe che estende Pytorch Dataset e definisce come le singole row sono estratte dal dataset.
In pratica ogni row è costituita da una input di lunghezza max_lenght e un target costituito dal prossimo carattere

In [29]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        # Tokenizes the entire text 
        token_ids = tokenizer.encode(txt)  

        # Uses a sliding window to chunk the book into overlapping sequences of max_length 
        for i in range(0, len(token_ids) - max_length, stride):    
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    # Returns the total number of rows in the dataset 
    def __len__(self):  
        return len(self.input_ids)

    # Returns a single row from the dataset 
    def __getitem__(self, idx):         #4
        return self.input_ids[idx], self.target_ids[idx]


/home/michele/dev/python/build_GPT/.venv/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


Questo dataset verrà utilizzato per caricare i batch attraverso l'utilizzo di un Pytorch DataLoader.

A tal fine definisco una funzione che genera i batch con le copie input e target

In [30]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    # Initializes the tokenize
    tokenizer = tiktoken.get_encoding("gpt2")                       
    # Creates dataset 
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)   
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        # drop_last=True drops the last batch if it is shorter than the specified batch_size to prevent loss spikes during training. 
        drop_last=drop_last,     
        # The number of CPU processes to use for preprocessing 
        num_workers=num_workers    
    )

    return dataloader


Per testare questa funzione provo a invocarla con batch_size = 1 e context_size = 4

In [31]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

#  Converts dataloader into a Python iterator to fetch the next entry via Python’s built-in next() function 
data_iter = iter(dataloader)     
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


Il primo tensor contiene gli input token id mentre il secondo i target token id.

Provo a invocare un nuovo batch per verificare a cosa serve il parametro stride (permette di implementare lo slinding window)

In [32]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


Per vedere l'utilizzo del dataloader con batch_size = 4 

In [33]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# EMBEDDING
Ultimo step della fase di preparazione dei dati per il training di un LLM è la conversione dei token id in embedding vectors. A tal fine è necessario inizializzare gli embeddings con valori random.

Di seguito vediamo un esempio per capire come funziona il processo di embedding

In [34]:
# Definisco un tensor di input token
input_ids = torch.tensor([2, 3, 5, 1])
#definisco la dimensione del vocab
vocab_size = 6
#definisco la dimensione dell'embedding
output_dim = 3
# Creo un ambedding layer con le dimensione sopra definite
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


Questi sono valori random che dovranno essere ottimizzati col processo di training del LLM

Ora, per ottenere l'embedding del token id 3

In [35]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In pratica l'embedding layer è una operazione di lookup. 

Per ottenere gli embeddings dei token id di esempio è quindi sufficiente passarli in input all'embedding layer

In [38]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


Gli embedding così ottenuti sono però indipendenti dalla posizione del token id all'interno della sequenza di input.<br>
Dal momento che anche il meccanismo di self attention è indipendente dalla posizione, è quindi necessario prevedere di aggiungere all'embedding l'informazione relativa alla posizione.

A tal fine si può utilizzare:
- relative positional embeddings
- absolute positional embeddings
Entrambi hanno l'obiettivo di aumentare la capacità dell'LLM di capire l'ordine e le relazioni tra i token

GPT utilizza absolte positiona encoding che viene ottimizzata durante il processo di training.
Tornando quindi alla dimensione del vocab del nostro BPE Tokenizer e con una dimensione di embedding di 256

In [39]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

A questo punto è possibile utilizzare il dataloader precedentemente creato e passare al token_embedding_layer dei batch di token_id.
Dopo aver estratto i batch avremo un tensor di dimensione (batch_size, max_lenght)

In [42]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:\n torch.Size([8, 4])


 Questo tensor di token_id viene passato all'embedding layer per convertire i token_id nei corrispondenti embeddings.<br> In questo modo avremo in output un tensor in formato (Batch_size, sequence_lenght, output_dim)

In [43]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


Per gestire il positional embedding (absolute) devo utilizzare un'altro embedding layer con la stessa embedding dimension (output_dim)

In [44]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


Questo positional embedding tensor può essere aggiunto al token embedding tensor

In [48]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In questo modo si conclude il processo di preparazione dei dati per il training del LLM. <BR>
In input al LLM andrà quindi un tensor di dimensione (Batch, context, Embedding dim)